In [19]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 34.3 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 596 kB 54.6 MB/s 
     |████████████████████████████████| 6.6 MB 57.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [116]:
train_df1 = pd.read_csv('/content/drive/MyDrive/vax_train.csv')

In [117]:
train_df1.head()


,id,tweet,label
0,1329771561671417856,Today Pfizer said it's asking U.S. regulators ...,Neutral
1,1266030447919804420,Good news on development of new #covid19 #vacc...,ProVax
2,1238769307456172034,Once they get a corona vaccine I don’t want to...,ProVax
3,1328814394822705152,Free flu vaccines and COVID-19 tests! https://...,Neutral
4,1254610119876280320,"Bollywood Gupshup,Russian Researchers Will Tes...",ProVax


In [118]:
train_df1['ProVax'] = 0

In [119]:
train_df1['Neutral'] = 0
train_df1['AntiVax'] = 0

In [120]:
print(train_df1.loc[[2]]['label'] == 'ProVax')

2    True
Name: label, dtype: bool


In [121]:
train_df1['ProVax'] = train_df1.apply(lambda row: 1 if row.label == 'ProVax' else 0, axis=1)


In [122]:
train_df1['AntiVax'] = train_df1.apply(lambda row: 1 if row.label == 'AntiVax' else 0, axis=1)
train_df1['Neutral'] = train_df1.apply(lambda row: 1 if row.label == 'Neutral' else 0, axis=1)
# train_df1['ProVax'] = train_df1.apply(lambda row: 1 if row.label == 'ProVax' else 0, axis=1)


In [123]:
train_df1.head()

,id,tweet,label,ProVax,Neutral,AntiVax
0,1329771561671417856,Today Pfizer said it's asking U.S. regulators ...,Neutral,0,1,0
1,1266030447919804420,Good news on development of new #covid19 #vacc...,ProVax,1,0,0
2,1238769307456172034,Once they get a corona vaccine I don’t want to...,ProVax,1,0,0
3,1328814394822705152,Free flu vaccines and COVID-19 tests! https://...,Neutral,0,1,0
4,1254610119876280320,"Bollywood Gupshup,Russian Researchers Will Tes...",ProVax,1,0,0


In [124]:
train_df1.drop('label', axis=1, inplace=True)

In [125]:
train_df1.head()

,id,tweet,ProVax,Neutral,AntiVax
0,1329771561671417856,Today Pfizer said it's asking U.S. regulators ...,0,1,0
1,1266030447919804420,Good news on development of new #covid19 #vacc...,1,0,0
2,1238769307456172034,Once they get a corona vaccine I don’t want to...,1,0,0
3,1328814394822705152,Free flu vaccines and COVID-19 tests! https://...,0,1,0
4,1254610119876280320,"Bollywood Gupshup,Russian Researchers Will Tes...",1,0,0


In [126]:
target_list = ['ProVax',	'Neutral',	'AntiVax']

In [76]:
# for i in range(4392):
#   if(train_df.loc[[i]]['label'].values == 'ProVax'):
#     train_df.loc[[i]]['ProVax'] = 1
#   elif(train_df.loc[[i]]['label'].values == 'AntiVax'):
#     train_df.loc[[i]]['AntiVax'] = 1
#   else:
#     train_df.loc[[i]]['Neutral'] = 1


In [13]:
# train_df['label']=train_df['label'].apply(lambda x: 1 if x=='ProVax' else 2 if x=='AntiVax' else 0 )


In [127]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-05

In [128]:
from transformers import BertTokenizer, BertModel


In [129]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [130]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['tweet']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        ) 

        return {
            
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }      





In [131]:
train_size = 0.8
train_df = train_df1.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = train_df1.drop(train_df.index).reset_index(drop=True)

In [132]:
train_df

,id,tweet,ProVax,Neutral,AntiVax
0,1300903006762930177,If I needed yet another reason to be disgusted...,1,0,0
1,1326261919200796674,If that coronavirus vaccine makes me grow a ne...,1,0,0
2,1244830704384364544,* And lastly since the cases of COVID-19 are o...,1,0,0
3,1325849813297016833,I hope @pfizer puts warning that your size wil...,0,0,1
4,1333996136697188352,@MaryKretzmann @svacheer1999 @EddieUsedYawn @J...,1,0,0
...,...,...,...,...,...
3509,1303057146381250560,Biden would take a COVID vaccine TOMORROW even...,1,0,0
3510,1325940903563964416,If it's true that Pfizer have developed a vacc...,0,0,1
3511,1329062095116247041,"With the usual fetal tissues, this is the pack...",0,0,1
3512,1339980867003936769,If you ever put Matter Reshaper into your mode...,1,0,0


In [133]:
val_df

,id,tweet,ProVax,Neutral,AntiVax
0,1331155378143338498,COVID-19 vaccine that it is entirely in the ha...,0,1,0
1,1325716182419312641,How they’ll know if the COVID-19 vaccines work...,0,1,0
2,1328315518495887361,SNEWZ UPDATE: Moderna says its coronavirus vac...,0,1,0
3,1329213362685812739,@wenter2420 @DylanGiaccio @Enasis @NYGovCuomo ...,1,0,0
4,1260604824506322946,US Says China Trying to Steal COVID-19 Vaccine...,0,1,0
...,...,...,...,...,...
873,1326106718519128064,"👇👇Not a good practice, not good for public tru...",0,1,0
874,1326136835916095493,Something about this vaccine scares me! Anyone...,0,0,1
875,1326186592847982593,@TinkerChip @matthaig1 They have been working ...,1,0,0
876,1325792316716625920,Fingers crossed this is positive data.. \n\nPf...,1,0,0


In [134]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [135]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [136]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [137]:
print(device)

cuda


In [138]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)    

In [139]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 3)               #3 classes
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [140]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [141]:
val_targets=[]
val_outputs=[]

In [142]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))

    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      print(len(training_loader))
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [143]:
ckpt_path = "/content/drive/MyDrive/curr_ckpt"
best_model_path = "/content/drive/MyDrive/best_model.pt"

In [144]:
# print(len(training_loader))

In [145]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
110
Epoch: 1 	Avgerage Training Loss: 0.005494 	Average Validation Loss: 0.019149
Validation loss decreased (inf --> 0.019149).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
110
Epoch: 2 	Avgerage Training Loss: 0.004682 	Average Validation Loss: 0.016082
Validation loss decreased (0.019149 --> 0.016082).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
#########

In [292]:
example = 'Dont be complacent - young Pasifika urged to get vaccinated this weekend https://t.co/ahO54WqVkX https://t.co/Y7ECLdw4ew'

In [293]:
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

In [294]:
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)
    print(train_df.columns[2:].to_list()[int(np.argmax(final_output, axis=1))])

[[0.9880850911140442, 0.016035789623856544, 0.0037489254027605057]]
ProVax


In [261]:
df_again = pd.read_csv('/content/drive/MyDrive/vax_train.csv')

In [262]:
testset = df_again.sample(frac= 0.3, random_state=200).reset_index(drop=True)

In [263]:
testset['ProVax'] = testset.apply(lambda row: 1 if row.label == 'ProVax' else 0, axis=1)
testset['Neutral'] = testset.apply(lambda row: 1 if row.label == 'Neutral' else 0, axis=1)
testset['AntiVax'] = testset.apply(lambda row: 1 if row.label == 'AntiVax' else 0, axis=1)



In [264]:
testset.head()

,id,tweet,label,ProVax,Neutral,AntiVax
0,1300903006762930177,If I needed yet another reason to be disgusted...,ProVax,1,0,0
1,1326261919200796674,If that coronavirus vaccine makes me grow a ne...,ProVax,1,0,0
2,1244830704384364544,* And lastly since the cases of COVID-19 are o...,ProVax,1,0,0
3,1325849813297016833,I hope @pfizer puts warning that your size wil...,AntiVax,0,0,1
4,1333996136697188352,@MaryKretzmann @svacheer1999 @EddieUsedYawn @J...,ProVax,1,0,0


In [265]:
testset['tweet'][4]

"@MaryKretzmann @svacheer1999 @EddieUsedYawn @JeremyBleez @CEOPRODIGY @GavinNewsom Herd immunity isn't worth talking about without a vaccine because of the ENORMOUS number of people that would have to get it. Remedies for COVID? Tell me more."

In [266]:
testset['prediction'] = ''

In [267]:
testset.head()

,id,tweet,label,ProVax,Neutral,AntiVax,prediction
0,1300903006762930177,If I needed yet another reason to be disgusted...,ProVax,1,0,0,
1,1326261919200796674,If that coronavirus vaccine makes me grow a ne...,ProVax,1,0,0,
2,1244830704384364544,* And lastly since the cases of COVID-19 are o...,ProVax,1,0,0,
3,1325849813297016833,I hope @pfizer puts warning that your size wil...,AntiVax,0,0,1,
4,1333996136697188352,@MaryKretzmann @svacheer1999 @EddieUsedYawn @J...,ProVax,1,0,0,


In [270]:
# testset.columns = 
testset = testset[['id',	'tweet','prediction','label',	'ProVax'	,'Neutral',	'AntiVax']]

In [271]:
testset.head()

,id,tweet,prediction,label,ProVax,Neutral,AntiVax
0,1300903006762930177,If I needed yet another reason to be disgusted...,,ProVax,1,0,0
1,1326261919200796674,If that coronavirus vaccine makes me grow a ne...,,ProVax,1,0,0
2,1244830704384364544,* And lastly since the cases of COVID-19 are o...,,ProVax,1,0,0
3,1325849813297016833,I hope @pfizer puts warning that your size wil...,,AntiVax,0,0,1
4,1333996136697188352,@MaryKretzmann @svacheer1999 @EddieUsedYawn @J...,,ProVax,1,0,0


In [272]:
for i in range(1318):
  encodings = tokenizer.encode_plus(
    testset['tweet'][i],
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
  )

  model.eval()
  with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)
    print(testset.columns[4:].to_list()[int(np.argmax(final_output, axis=1))])
    testset['prediction'][i] = testset.columns[4:].to_list()[int(np.argmax(final_output, axis=1))]


  



[[0.989891529083252, 0.008121231570839882, 0.008276371285319328]]
ProVax
[[0.995145857334137, 0.014088490046560764, 0.007951410487294197]]
ProVax
[[0.9910492897033691, 0.012692724354565144, 0.004919787868857384]]
ProVax
[[0.01713975891470909, 0.019473684951663017, 0.984211266040802]]
AntiVax
[[0.9888399243354797, 0.012767923064529896, 0.0064241052605211735]]
ProVax
[[0.0295451357960701, 0.013333970680832863, 0.9877653121948242]]
AntiVax
[[0.03561047092080116, 0.9825329780578613, 0.007319541648030281]]
Neutral
[[0.02697872929275036, 0.012732714414596558, 0.9896281957626343]]
AntiVax


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[[0.026442447677254677, 0.013520180247724056, 0.9883623719215393]]
AntiVax
[[0.021849682554602623, 0.07065482437610626, 0.8334174752235413]]
AntiVax
[[0.005762496963143349, 0.9881049394607544, 0.026164032518863678]]
Neutral
[[0.010950426571071148, 0.989795982837677, 0.010452492162585258]]
Neutral
[[0.01182439737021923, 0.9873051047325134, 0.008270099759101868]]
Neutral
[[0.9925991296768188, 0.012357707135379314, 0.006554299965500832]]
ProVax
[[0.9780760407447815, 0.03141336888074875, 0.003340264316648245]]
ProVax
[[0.02144474908709526, 0.015344454906880856, 0.9896290898323059]]
AntiVax
[[0.9901124835014343, 0.01171945035457611, 0.005368602462112904]]
ProVax
[[0.02786155790090561, 0.01321791298687458, 0.9902257323265076]]
AntiVax
[[0.9958190321922302, 0.009749311953783035, 0.008061441592872143]]
ProVax
[[0.9954097867012024, 0.009409072808921337, 0.008381483145058155]]
ProVax
[[0.9950820207595825, 0.006224394775927067, 0.00883570034056902]]
ProVax
[[0.01870386302471161, 0.987249791622161

In [ ]:
# testset['temp'] = testset['prediction']
# testset['prediction'] = testset['label']



In [273]:
testset.tail(20)

,id,tweet,prediction,label,ProVax,Neutral,AntiVax
1298,1325926513594818562,Isn’t this the bit that a villain intent on wo...,AntiVax,AntiVax,0,0,1
1299,1250977545689788416,Ok im going to take a chance on @GileadScience...,Neutral,Neutral,0,1,0
1300,1325985742569476098,"Sorry, Not taking any vaccine no matter how mu...",AntiVax,AntiVax,0,0,1
1301,1327800753826660352,States waiting for answers from feds on corona...,Neutral,Neutral,0,1,0
1302,1334781218978746370,@simsy1977 I have been to work every day in co...,AntiVax,AntiVax,0,0,1
1303,1331341189279338497,@newtgingrich This vaccine for the coronavirus...,Neutral,Neutral,0,1,0
1304,1335489340990951424,Joe Biden says he won't mandate getting COVID-...,Neutral,Neutral,0,1,0
1305,1327367449709309954,"@TeamTrump @realDonaldTrump Define ""we""... Cer...",ProVax,ProVax,1,0,0
1306,1325776898782158850,Pfizer vaccine's early data shows 90% effectiv...,ProVax,ProVax,1,0,0
1307,1326070843030474753,Pfizer's COVID-19 Vaccine News Throws Bitcoin ...,Neutral,Neutral,0,1,0


In [274]:
from sklearn.metrics import accuracy_score

accuracy_score(testset['prediction'], testset['label'])

0.9946889226100152

In [ ]:
# model.eval()
# with torch.no_grad():
#     input_ids = encodings['input_ids'].to(device, dtype=torch.long)
#     attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
#     token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
#     output = model(input_ids, attention_mask, token_type_ids)
#     final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
#     print(train_df.columns[2:].to_list()[int(np.argmax(final_output, axis=1))])

In [275]:
test_data = pd.read_csv('/content/drive/MyDrive/vax_test.csv')

In [277]:
test_data.tail()

,id,tweet
712,1426222975108407296t,@DrunkenDucksFan @maryloupatey @CP24 https://t...
713,1471836133613084672t,"@JustinTrudeau @fordnation Justin, how can you..."
714,1472586840301617153t,@ThatAmish1 Have they seen all of the sporting...
715,1423061697556668420t,@mimij1992 @SClarified @OregonGovBrown @OHAOre...
716,1493703003400019976t,@KevinPIvey @Tracinski Because the difference ...


In [279]:
test_data['prediction'] = ''

In [280]:
test_data.head()

,id,tweet,prediction
0,1472958183882719233t,So true. Take control of what you can…get vac...,
1,1449995880904470529t,The vaccinated still spread it and are factori...,
2,1480413964265660416t,"NT records 403 new cases of COVID-19, as it ex...",
3,1408119497962057742t,@ochealth You are experimenting with people. B...,
4,1439361704320806915t,@LPCPressBox So I guess we’re supposed to just...,


In [282]:
test_data['ProVax'] = ''
test_data['Neutral'] = ''
test_data['AntiVax'] = ''

In [283]:
test_data.head()

,id,tweet,prediction,ProVax,Neutral,AntiVax
0,1472958183882719233t,So true. Take control of what you can…get vac...,,,,
1,1449995880904470529t,The vaccinated still spread it and are factori...,,,,
2,1480413964265660416t,"NT records 403 new cases of COVID-19, as it ex...",,,,
3,1408119497962057742t,@ochealth You are experimenting with people. B...,,,,
4,1439361704320806915t,@LPCPressBox So I guess we’re supposed to just...,,,,


In [284]:
for i in range(717):
  encodings = tokenizer.encode_plus(
    test_data['tweet'][i],
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
  )

  model.eval()
  with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)
    print(test_data.columns[3:].to_list()[int(np.argmax(final_output, axis=1))])
    test_data['prediction'][i] = test_data.columns[3:].to_list()[int(np.argmax(final_output, axis=1))]


  



[[0.7475327849388123, 0.006559872534126043, 0.31508228182792664]]
ProVax
[[0.3302147686481476, 0.011105535551905632, 0.4001104235649109]]
AntiVax
[[0.011619135737419128, 0.9905912280082703, 0.01186444703489542]]
Neutral
[[0.7086015343666077, 0.023572340607643127, 0.10351932048797607]]
ProVax
[[0.04988957569003105, 0.9813854694366455, 0.005765136331319809]]
Neutral
[[0.9891786575317383, 0.01838395930826664, 0.003923365846276283]]
ProVax
[[0.9841992855072021, 0.005799815058708191, 0.018240930512547493]]
ProVax
[[0.4251200258731842, 0.0031190284062176943, 0.7651395201683044]]
AntiVax
[[0.022654518485069275, 0.9877382516860962, 0.011162998154759407]]
Neutral
[[0.03831310197710991, 0.979951798915863, 0.0036500284913927317]]
Neutral
[[0.11351156234741211, 0.005063461139798164, 0.9676607251167297]]
AntiVax
[[0.97455894947052, 0.007026149425655603, 0.027265941724181175]]
ProVax
[[0.5986534953117371, 0.38395166397094727, 0.003171929158270359]]
ProVax
[[0.06540119647979736, 0.009465622715651989,

In [300]:
test_data.head(42)

,id,tweet,prediction,ProVax,Neutral,AntiVax
0,1472958183882719233t,So true. Take control of what you can…get vac...,ProVax,,,
1,1449995880904470529t,The vaccinated still spread it and are factori...,AntiVax,,,
2,1480413964265660416t,"NT records 403 new cases of COVID-19, as it ex...",Neutral,,,
3,1408119497962057742t,@ochealth You are experimenting with people. B...,ProVax,,,
4,1439361704320806915t,@LPCPressBox So I guess we’re supposed to just...,Neutral,,,
5,1478769637282230274t,Vaccinated and Healthy. Dm quick. https://t.co...,ProVax,,,
6,1472297934633017344t,@MedelliaS @KrispyKrunch17 @GijsDongen @matthe...,ProVax,,,
7,1520114105985769475t,@SheltsUnited @ChristianBrie1 @EssexPR There i...,AntiVax,,,
8,1423827464573255683t,Employees will get thousands of dollars if the...,Neutral,,,
9,1479557209751175174t,Pfizer/BioNTech vaccine protects children agai...,Neutral,,,


In [302]:
test_data.drop(['tweet', 'ProVax'	,'Neutral',	'AntiVax'], axis = 1, inplace = True)

In [307]:
test_data.head()

,id,pred
0,1472958183882719233t,ProVax
1,1449995880904470529t,AntiVax
2,1480413964265660416t,Neutral
3,1408119497962057742t,ProVax
4,1439361704320806915t,Neutral


In [306]:
test_data.columns = ['id', 'pred']

In [309]:
test_data.to_csv('akshit_rohit.csv', header = True, index = False)